# Day 3


In [3]:
import re
import numpy as np

In [196]:
class EngineSchematic:
    def __init__(self, filename: str):
        self.schematic = self.get_engine_schematic(filename)
        self.previous_line = None
        self.current_line = None
        self.next_line = None
        self.engine_numbers = []
        self.engine_sum = 0
        self.symbols = []
        self.checked_positions = []
        
    @staticmethod 
    def get_engine_schematic(filename: str):
        f = open(f'{filename}.txt', 'r')
        f.seek(0)
        schematic = np.array(["." + line.strip() + "." for line in f.readlines()])
        return schematic
    
    def process_engine_schematic(self):
        for index, line in enumerate(self.schematic, start=0):
            self.symbols = []
            self.previous_line = self.schematic[index - 1] if index > 0 else None
            self.current_line = line
            self.next_line = self.schematic[index + 1] if index < len(self.schematic) - 1 else None
            
            self.process_lines()
            
        for number in self.engine_numbers:
            self.engine_sum += number
            
        print(self.engine_sum)
            
    def process_lines(self):
        numbers = re.finditer(r"\d+", self.current_line)
        
        lines = [line for line in [self.previous_line, self.current_line, self.next_line] if line is not None]
        
        for index, number_match in enumerate(numbers):
            number = int(number_match.group())
            start, end = number_match.span()
            for line in lines:
                for char in line[start-1:end+1]:
                    if char != "." and not char.isdigit():
                        self.engine_numbers.append(number)
                        break

    def get_gear_ratios(self):
        for index, line in enumerate(self.schematic, start=0):
            self.previous_line = self.schematic[index - 1] if index > 0 else None
            self.current_line = line
            self.next_line = self.schematic[index + 1] if index < len(self.schematic) - 1 else None

            self.find_gears()

        for number in self.engine_numbers:
            self.engine_sum += number

        print(self.engine_sum)

    def find_gears(self):
        lines = [line for line in [self.previous_line, self.current_line, self.next_line] if line is not None]
        symbols = re.finditer(r"\*", self.current_line)

        for symbol in symbols:
            position = symbol.start()
            numbers = []
            for line in lines:
                numbers_in_line = re.finditer(r"\d+", line)
                for number in numbers_in_line:
                    if position - 1 <= number.start() <= position + 1 or position - 1 <= number.end() - 1 <= position + 1:
                        numbers.append(int(number.group()))
            
            if len(numbers) == 2:
                n1, n2 = numbers
                self.engine_numbers.append(n1 * n2)

## Pt 1


In [185]:
schematic_sum = EngineSchematic(filename="input")
schematic_sum.process_engine_schematic()

549908


## Pt 2

In [198]:
total_gear_ratio = EngineSchematic(filename="input")
total_gear_ratio.get_gear_ratios()

81166799
